In [ ]:
%matplotlib inline
from collections import defaultdict
import pymc3 as pm
from pymc3 import traceplot
import numpy as np
from matplotlib import pyplot as plt
import json
from pymongo import MongoClient
import pymongo
from bson.objectid import ObjectId

## Data

### Fetching the data

In [ ]:
client = MongoClient()
db = client.heroku_rnwkcq9r

In [ ]:
def question_text(question_id):
    question = db.question.find_one({'_id': question_id})
    return question['text']

In [ ]:
def question_max_value(question_id):
    question = db.question.find_one({'_id': question_id})
    if question['question_type'] == "boolean":
        return 1
    elif question["question_type"] == "numerical":
        if 'numericalAnswers' in question:
            max_value = max(map(int,question['numericalAnswers'].keys()))
            return max_value
        else:
            return 5

In [ ]:
def answer_value(answer_id):
    answer = db.answer.find_one({'_id': answer_id})
    q_id = answer['question']
    if 'numerical_answer' in answer:
        return answer['numerical_answer'] / float(question_max_value(q_id))
    elif 'boolean_answer' in answer:
        return answer['boolean_answer'] / float(question_max_value(q_id))

In [ ]:
def answer_with_score(answer_id):
    answer = db.answer.find_one({'_id': answer_id})
    return 'text_answer' not in answer

In [ ]:
def score_handin_grader(handin_id,grader_id):
    # find all answers to the hand in of the grader and average it out
    report_grade = db.report_grade.find_one({'handin': handin_id, 'giver': grader_id})
    answers = db.answer.find({'report_grade': report_grade['_id'] })
    handin_acc = 0.0
    for answer in answers:
        if answer_with_score(answer['_id']):
            handin_acc = handin_acc + answer_value(answer['_id'])
    return handin_acc / float(answers.count())

In [ ]:
def score_handin_list(handin_id):
    scores = list()
    report_grades = db.report_grade.find({'handin': handin_id, 'state': 'ANSWERED'})
    for report_grade in report_grades:
        scores.append((score_handin_grader(handin_id,report_grade['giver']),report_grade['giver']))
    return scores

In [ ]:
# finds the reported score for each handins in an assignment
# returns a list containing the score of each handin
def assignment_handins(assignment_id):
    handins_reports = list()
    handins = db.handin.find({'assignment': assignment_id})
    for handin in handins:
        handins_reports.append((score_handin_list(handin['_id']),handin['_id']))
    return handins_reports

In [ ]:
def grader_ids(assignment_id):
    graders = set()
    report_grades = db.report_grade.find({'assignment':assignment_id})
    for report_grade in report_grades:
        graders.add(report_grade['giver'])
    return list(graders)

In [ ]:
def grader_name(grader_id):
    grader = db.user.find_one({'_id': grader_id})
    return grader['name']

In [ ]:
def course_handins(course_id):
    handins = list()
    assignments = db.assignment.find({'course':course_id})
    for assignment in assignments:
        handins.extend(assignment_handins(assignment['_id']))
    return handins

In [ ]:
def find_bias(assignment_id,mcmc,f):
    bias_abs = list()
    for g in grader_ids(assignment_id):
        try:     
            bias_mean = f(mcmc.trace('B_%s' % str(g))[:])
            name = str(g)
            bias_abs.append((abs(bias_mean),name))
        except:
            print "error"
            pass
    return bias_abs

## Models

In [ ]:
data = [(0.2726495726495726, ObjectId('55e4330ded7a0c0009f0d9d9')),
 (0.6410256410256411, ObjectId('55f2d136d8786100099ff332')),
 (0.5128205128205128, ObjectId('55db3833edf3950009412132')),
 (0.46153846153846156, ObjectId('55e59e8336cc7d00092c414a')),
 (0.6410256410256411, ObjectId('55d9cb9ea6fb8f00080da473')),
 (0.6153846153846154, ObjectId('55db3833edf395000941213d')),
 (0.4871794871794871, ObjectId('56056a8689c698000f9c29bd')),
 (0.9957264957264957, ObjectId('55db3832edf395000941211a'))]

In [ ]:
data = map(lambda x: x[0],data)

In [ ]:
data

In [ ]:
with pm.Model() as s_model:
    N = len(data)

    # True score
    T_mu = pm.Uniform('T_mu',0,1)
    T = pm.Normal('T', mu=T_mu, tau=100, testval = np.mean(data))

    # Bias
    B = list()
    O = list()

    for i in range(0, N):
        B.append(pm.Normal('B_%i' % i, mu=0, tau=100, testval=0))
        O.append(pm.Normal('O_%i' % i, mu = T + B[i], tau=100, observed=data[i]))

    distr = [T_mu, T]
    distr.extend(B)
    distr.extend(O)
    step = pm.NUTS(distr)

In [ ]:
with s_model:
    start = pm.find_MAP()
    mcmc = pm.sample(2000, step, start=start)

In [ ]:
T = mcmc["T"][:]
plt.hist(T, histtype="step", normed=True, alpha=0.5, bins=30,
         label="T", color="#348ABD")
B0 = mcmc["B_1"][:]
plt.hist(B0, histtype="step", normed=True, alpha=0.5, bins=30,
         label="B0")
plt.xlim(-0.5,1)
plt.legend()
plt.show()

In [ ]:
print "The mean value of the observed data:"
print np.mean(data)
print "The expected value of the normal distribution of T, the true score:"
print np.mean(T)
print "The expected value of the normal distribution of the bias of the first grader B0:"
print np.mean(B0)

In [ ]:
data_handin = assignment_handins(ObjectId("55f277cff3bd61000a2112d4"))

In [ ]:
bias_abs_1 = find_bias(ObjectId("55f277cff3bd61000a2112d4"), mcmc_handin, np.mean)
bias_abs_2 = find_bias(ObjectId("55f277cff3bd61000a2112d4"), mcmc_handin2, np.mean)
bias_abs_1.sort()
bias_abs_2.sort()

In [ ]:
compared_12 = list()
for idx, (value, _id) in enumerate(bias_abs_1):
    for item in enumerate(bias_abs_2):
        if item[1][1] == _id:
            compared_12.append((_id ,idx, item[0]))

In [ ]:
ax = plt.subplot(111)

largest_diff = 0
largest_diff_id = ""

for idx, (_id, bias1,bias2) in enumerate(compared_12):
    if bias1 > bias2:
        plt.plot([idx,idx],[bias1,bias2], color='g')
    else:
        plt.plot([idx,idx],[bias1,bias2], color='r')
    if abs(bias1-bias2) > largest_diff:
        largest_diff = abs(bias1-bias2)
        largest_diff_id = _id

box = ax.get_position()
ax.set_position([box.x0, box.y0,
                 box.width * 3, box.height])
    
plt.show()

In [ ]:
data = course_handins(ObjectId("55d9cb9ea6fb8f00080da4a1"))

In [ ]:
def handin_model(model, data):
    with pm.Model() as model:
        N_H = len(data)

        # Bias
        T = dict()
        B = dict()
        O = list()

        for h in range(0, N_H):
            (scores,h_id) = data[h]
            N_G = len(scores)

    #        T_temp = pm.Uniform('T_mu_%s' % str(h_id),lower=0,upper=1,testval = np.mean(map(lambda x:x[0],scores)))
    #        T_mu.append(T_temp)
            T[h_id] = pm.Normal('T_%s' % str(h_id), mu=np.mean(map(lambda x:x[0],scores)), tau=100)

            for g in range(0, N_G):
                (val,g_id) = scores[g]
                if g_id not in B:
                    B[g_id] = pm.Normal('B_%s' % str(g_id), mu=0, tau=100)
                O.append(pm.Normal('O_%(h)i_%(g)i' % {'h': h, 'g':g}, mu = T[h_id] + B[g_id], tau=100, observed=val))

        distr = list()
        distr.extend(T.values())
        distr.extend(B.values())
        distr.extend(O[:])
        step = pm.NUTS(distr)
        return step
        
def execute_model(model,step):
    with model:
        start = pm.find_MAP()
        mcmc = pm.sample(2000, step, start=start)
        return mcmc

In [ ]:
c_model = pm.Model()

In [ ]:
step = handin_model(c_model,data)

In [ ]:
mcmc1 = execute_model(c_model,step)

In [ ]:
mcmc2 = execute_model(c_model,step)